In [35]:
import os
from pathlib import Path
import string
import numpy

import pandas
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
articles_dir = Path(os.getcwd(), 'data', 'articles')
titles_dir = Path(os.getcwd(), 'data', 'titles')

In [33]:
NUM_WORDS = 2 ** 16
MAX_TEXT_LENGTH = 42

# Data preprocessing (word embedding)

In [46]:
# Remove puncuation from text
def remove_punc_from_text(text):
    clean_word_list = [char for char in text if char not in string.punctuation]
    return ''.join(clean_word_list)


# TODO: remove stopwords

In [47]:
# Collect articles content and summary to dataframe
dataset = []
list_article_files = os.listdir(articles_dir)
for filename in list_article_files:
    with open(Path(articles_dir, filename), 'r') as article_file:
        text = article_file.read()
        with open(Path(titles_dir, filename), 'r') as title_file:
            summary = title_file.read()
            dataset.append({ 'summary': summary, 'text': text })

dataset_df = pandas.DataFrame.from_records(dataset)

# Normalize train data
dataset_df.summary = [f'_START_{remove_punc_from_text(summary.lower())}_END_' for summary in dataset_df['summary']]
dataset_df.text = [remove_punc_from_text(text.lower()) for text in dataset_df['text']]

# Split datasets to training and validation sets
x_train, x_val, y_train, y_val = train_test_split(
    numpy.array(dataset_df['text']),
    numpy.array(dataset_df['summary']),
    test_size=0.1,
    random_state=1,
    shuffle=True
)

# Tokenizing text of x set
x_tokenizer = Tokenizer(num_words=NUM_WORDS)
x_tokenizer.fit_on_texts(list(x_train))
x_train_sequence = x_tokenizer.texts_to_sequences(x_train)
x_train_padded = pad_sequences(x_train_sequence, maxlen=MAX_TEXT_LENGTH, padding='post')
x_vocab_size = len(x_tokenizer.word_index) + 1

# Tokenizing text of y set
y_tokenizer = Tokenizer(num_words=NUM_WORDS)
y_tokenizer.fit_on_texts(list(y_train))
y_train_sequence = x_tokenizer.texts_to_sequences(y_train)
y_train_padded = pad_sequences(y_train_sequence, maxlen=MAX_TEXT_LENGTH, padding='post')
y_vocab_size = len(y_tokenizer.word_index) + 1

print(f'[Vocab size] x: {x_vocab_size} <-> y: {y_vocab_size}')

[Vocab size] x: 25966 <-> y: 3803


# Modeling

In [48]:
LATENT_DIM = 240
EMBEDDING_DIM = 300
NUM_EPOCHS = 50

GLOVE_FILE_PATH = Path(os.getcwd(), 'glove', '')

In [49]:
def get_embedding_matrix (tokenizer, embedding_dim, vocab_size):
    word_index = tokenizer.word_index
    vocab = list(word_index.keys())

    with open(GLOVE_FILE_PATH) as file:
        for line in file:
            word, coefs = line.split(maxsplit=1)
            print(word, coefs)

    return 1


x_embedding_matrix = get_embedding_matrix(x_tokenizer, EMBEDDING_DIM, x_vocab_size)
y_embedding_matrix = get_embedding_matrix(y_tokenizer, EMBEDDING_DIM, y_vocab_size)

# print(x_embedding_matrix.shape)
# print(y_embedding_matrix.shape)

{'trong': 1, 'và': 2, 'của': 3, 'tăng': 4, 'với': 5, 'có': 6, 'các': 7, 'giá': 8, 'đầu': 9, 'thị': 10, 'là': 11, 'giảm': 12, 'trường': 13, 'phiếu': 14, 'cổ': 15, 'đồng': 16, 'dịch': 17, 'cho': 18, 'phiên': 19, 'giao': 20, 'điểm': 21, 'tư': 22, 'tỷ': 23, 'công': 24, 'năm': 25, 'được': 26, 'đã': 27, 'ngày': 28, 'hơn': 29, 'động': 30, 'mức': 31, 'số': 32, 'chứng': 33, 'nhà': 34, 'sẽ': 35, 'chỉ': 36, 'doanh': 37, 'đang': 38, 'từ': 39, 'trên': 40, 'hàng': 41, 'đó': 42, 'này': 43, 'khi': 44, 'lại': 45, 'hiện': 46, 'thể': 47, 'định': 48, 'về': 49, 'ở': 50, 'mạnh': 51, 'không': 52, 'theo': 53, 'chính': 54, 'hợp': 55, 'nhất': 56, 'bán': 57, 'tiếp': 58, 'những': 59, 'vào': 60, 'mua': 61, 'khoán': 62, 'sản': 63, 'gần': 64, 'mã': 65, 'tại': 66, 'hạn': 67, 'nhiều': 68, 'tháng': 69, 'đến': 70, 'lượng': 71, 'cũng': 72, 'một': 73, 'triệu': 74, 'như': 75, 'sau': 76, 'khối': 77, 'để': 78, 'điều': 79, 'nghiệp': 80, 'thời': 81, 'tục': 82, 'quan': 83, 'ty': 84, 'việt': 85, 'tiền': 86, 'phát': 87, 'tích': 8